In [ ]:
# 필요한 모듈을 설치합니다.
!pip install langchain openai chromadb html2text tiktoken sentence_transformers

In [ ]:
# ChatGPT를 이용하기 위해서는 api key가 필요합니다.
# 필요한 api key는 https://platform.openai.com에서 무료로 발급 받을수 있습니다.
# Google 검색을 위해서는 Google api key와 cse id가 필요합니다.
# api key는 Google cloud에서 cse id는 https://programmablesearchengine.google.com 에서 무료로 발급받을 수 있습니다.
GOOGLE_CSE_ID = "..."
OPENAI_API_KEY = "..."
GOOGLE_API_KEY = "..."

In [ ]:
import os
import torch
from langchain.chat_models.openai import ChatOpenAI
from langchain.retrievers.web_research import WebResearchRetriever
from langchain.chains import RetrievalQAWithSourcesChain, LLMChain
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate

os.environ["GOOGLE_CSE_ID"] = GOOGLE_CSE_ID
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

class ConversationLLM:
  def __init__(self):
    self.llm = ChatOpenAI()
    self.search = GoogleSearchAPIWrapper()
    self.prompt = PromptTemplate.from_template("{question}")

  def set_chain(self, model_name):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model_kwargs = {'device': device}
    # Embedding 모델은 HuggingFace를 이용합니다.
    hf = HuggingFaceEmbeddings(
        model_name=model_name,
        model_kwargs=model_kwargs
    )

    # Chroma를 백터 스토리지로 사용합니다.
    vectorstore = Chroma(embedding_function=hf)
    # 검색결과가 임베딩되어 저장됩니다.
    web_search = WebResearchRetriever.from_llm(
        vectorstore=vectorstore,
        llm=self.llm,
        search=self.search
    )

    # RAG를 사용하는 체인과 아닌 체인을 각각 만듭니다.
    self.rag_chain = RetrievalQAWithSourcesChain.from_chain_type(self.llm, retriever=web_search)
    self.chain = LLMChain(llm=self.llm, prompt=self.prompt)

  # RAG 사용여부를 옵션으로 정합니다.
  def question(self, q, rag=False):
    return self.rag_chain({"question":q})['answer'] if rag else self.chain({"question":q})['text']

conversation_llm = ConversationLLM()
# HuggingFace에서 사용할 모델을 선택합니다.
# 여기서는 구글 다국어 Bert모델을 사용합니다.
conversation_llm.set_chain("bert-base-multilingual-uncased")

In [ ]:
# GPT3.5는 2023년 데이터가 없으므로 2023년 질문에 대답하지 못합니다.
q = "2023년 아시안게임 축구 금메달은 어느나라인지 말해줘"
conversation_llm.question(q, rag=False)

'2023년 아시안게임 축구 금메달은 아직 결정되지 않았으며, 예측할 수 없습니다. 아시안게임은 매번 다양한 국가들이 경기를 펼치기 때문에 누가 금메달을 따를지는 경기 진행 상황에 따라 결정됩니다.'

In [ ]:
# 검색으로 2023년 아시안게임 결과를 알려주니 잘 대답합니다.
conversation_llm.question(q, rag=True)

Fetching pages: 100%|##########| 1/1 [00:00<00:00,  2.19it/s]


'2023년 아시안게임 축구 금메달은 한국 대표팀입니다.\n'